In [1]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
productivity_prediction_of_garment_employees = fetch_ucirepo(id=597) 
  
# data (as pandas dataframes) 
X = productivity_prediction_of_garment_employees.data.features 
y = productivity_prediction_of_garment_employees.data.targets 
  
# metadata 
print(productivity_prediction_of_garment_employees.metadata) 
  
# variable information 
print(productivity_prediction_of_garment_employees.variables) 


{'uci_id': 597, 'name': 'Productivity Prediction of Garment Employees', 'repository_url': 'https://archive.ics.uci.edu/dataset/597/productivity+prediction+of+garment+employees', 'data_url': 'https://archive.ics.uci.edu/static/public/597/data.csv', 'abstract': 'This dataset includes important attributes of the garment manufacturing process and the productivity of the employees which had been collected manually and also been validated by the industry experts.', 'area': 'Business', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 1197, 'num_features': 14, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['actual_productivity'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2020, 'last_updated': 'Mon Feb 26 2024', 'dataset_doi': '10.24432/C51S6D', 'creators': [], 'intro_paper': {'ID': 399, 'type': 'NATIVE', 'title': 'Mining the productivity dat

In [3]:
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor
import warnings

from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier,CatBoostRegressor
from xgboost import XGBClassifier
import warnings

In [4]:
X.isnull().sum()

date                       0
quarter                    0
department                 0
day                        0
team                       0
targeted_productivity      0
smv                        0
wip                      506
over_time                  0
incentive                  0
idle_time                  0
idle_men                   0
no_of_style_change         0
no_of_workers              0
dtype: int64

In [5]:
y.isnull().sum()
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   actual_productivity  1197 non-null   float64
dtypes: float64(1)
memory usage: 9.5 KB


In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   day                    1197 non-null   object 
 4   team                   1197 non-null   int64  
 5   targeted_productivity  1197 non-null   float64
 6   smv                    1197 non-null   float64
 7   wip                    691 non-null    float64
 8   over_time              1197 non-null   int64  
 9   incentive              1197 non-null   int64  
 10  idle_time              1197 non-null   float64
 11  idle_men               1197 non-null   int64  
 12  no_of_style_change     1197 non-null   int64  
 13  no_of_workers          1197 non-null   float64
dtypes: float64(5), int64(5), object(4)
memory usage: 131.1+ 

In [7]:
X = X.drop('wip',axis=1)


In [8]:
X.head()

,date,quarter,department,day,team,targeted_productivity,smv,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,7080,98,0.0,0,0,59.0
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,960,0,0.0,0,0,8.0
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,3660,50,0.0,0,0,30.5
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,3660,50,0.0,0,0,30.5
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1920,50,0.0,0,0,56.0


In [9]:
X.set_index('date')

,quarter,department,day,team,targeted_productivity,smv,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
date,,,,,,,,,,,,
1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,7080,98,0.0,0,0,59.0
1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,960,0,0.0,0,0,8.0
1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,3660,50,0.0,0,0,30.5
1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,3660,50,0.0,0,0,30.5
1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1920,50,0.0,0,0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3/11/2015,Quarter2,finishing,Wednesday,10,0.75,2.90,960,0,0.0,0,0,8.0
3/11/2015,Quarter2,finishing,Wednesday,8,0.70,3.90,960,0,0.0,0,0,8.0
3/11/2015,Quarter2,finishing,Wednesday,7,0.65,3.90,960,0,0.0,0,0,8.0


In [10]:
#ohe = OneHotEncoder(sparse_output = False,handle_unknown = 'ignore').set_output(transform='pandas')


## CatBoost w/o OHE


In [12]:
cgbm = CatBoostRegressor(random_state=24, cat_features=list(X.columns[X.dtypes==object])) #can give like this -> ))
params = {
         'n_estimators': [10,20,30,50,100,200],
         'max_depth': [2,3,4],
         'learning_rate': np.linspace(0.001,1,6)         
}
          
          
         

kfold = KFold(n_splits = 5,shuffle = True,random_state =24)

In [13]:
gcv = GridSearchCV(cgbm,param_grid = params, cv = kfold, scoring = 'r2',verbose = 3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
0:	learn: 0.1586711	total: 152ms	remaining: 15.1s
1:	learn: 0.1524615	total: 167ms	remaining: 8.19s
2:	learn: 0.1462744	total: 188ms	remaining: 6.07s
3:	learn: 0.1444408	total: 210ms	remaining: 5.04s
4:	learn: 0.1400221	total: 225ms	remaining: 4.28s
5:	learn: 0.1356270	total: 245ms	remaining: 3.85s
6:	learn: 0.1330934	total: 266ms	remaining: 3.53s
7:	learn: 0.1313252	total: 286ms	remaining: 3.29s
8:	learn: 0.1293176	total: 306ms	remaining: 3.1s
9:	learn: 0.1281949	total: 327ms	remaining: 2.94s
10:	learn: 0.1264358	total: 346ms	remaining: 2.8s
11:	learn: 0.1256650	total: 368ms	remaining: 2.7s
12:	learn: 0.1245055	total: 390ms	remaining: 2.61s
13:	learn: 0.1242398	total: 412ms	remaining: 2.53s
14:	learn: 0.1238045	total: 433ms	remaining: 2.45s
15:	learn: 0.1237542	total: 456ms	remaining: 2.4s
16:	learn: 0.1224056	total: 477ms	remaining: 2.33s
17:	learn: 0.1208366	total: 498ms	remaining: 2.27s
18:	learn: 0.1204037	total: 518ms

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=<catboost.core.CatBoostRegressor object at 0x0000015F272240E0>,
             n_jobs=-1,
             param_grid={'learning_rate': array([0.001 , 0.2008, 0.4006, 0.6004, 0.8002, 1.    ]),
                         'max_depth': [2, 3, 4],
                         'n_estimators': [10, 20, 30, 50, 100, 200]},
             scoring='r2', verbose=3)

In [14]:
print(gcv.best_params_)
print(gcv.best_score_)

0.493967948722516

## LightGBM Boosting

In [16]:
# ohe = OneHotEncoder(sparse_output = False,handle_unknown = 'ignore').set_output(transform='pandas')
# ct_ohe = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
#                                   (ohe,make_column_selector(dtype_include = object) ),
#                                   verbose_feature_names_out = False).set_output(transform = 'pandas')
# X_ohe = ct_ohe.fit_transform(X)


In [17]:
from lightgbm import LGBMRegressor

In [18]:
lgbm = LGBMRegressor(random_state = 24)
params = {'LG__n_estimators': [10,20,30,50,100,200],
         'LG__max_depth': [2,3,4],
          'LG__learning_rate': np.linspace(0.001,1,12),
          'LG__max_features': [2,3,4,5,6],
          #'booster':['gbtree', 'gblinear'],
          #'num_parallel_tree':[2,3,4,5]
         }

kfold = KFold(n_splits = 5,shuffle = True,random_state =24)

In [19]:
ohe = OneHotEncoder(sparse_output = False,handle_unknown = 'ignore').set_output(transform='pandas')
ct_ohe = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                                  (ohe,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')

pipe  =Pipeline([('CT',ct_ohe),('LG',lgbm)])

In [20]:
gcv = GridSearchCV(pipe,param_grid = params, cv = kfold, scoring = 'r2',verbose = 3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 1080 candidates, totalling 5400 fits
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000056 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 332
[LightGBM] [Info] Number of data points in the train set: 1197, number of used features: 54
[LightGBM] [Info] Start training from score 0.735091
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000015F27225820>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transfo...
                                                          verbose_feature_names_out=False)),
                                       ('LG', LGBMRegressor(random_state=24))]),
             n_jobs=-1,
             param_grid={'LG__learning_rate': array([0.001     , 0.09181818, 0.18263636, 0.27345455, 0.36427273,
       0.45509091, 0.54590909, 0.63672727, 0.72754545, 0.81836364,
       0.90918182, 1.        ]),
                         'LG__max_depth': [2, 3, 4],
                         'LG__max_features': [2, 3, 4, 5, 6],
                         'LG__n_estimators': [10, 20, 30, 50, 100, 200]},
             scoring='r2', verbose=3)

In [21]:
print(gcv.best_params_)
print(gcv.best_score_)

0.5044833721340365

## XGBoost

In [23]:
from xgboost import XGBRegressor

xgbm = XGBRegressor(random_state = 24)

In [24]:
ohe = OneHotEncoder(sparse_output = False,handle_unknown = 'ignore').set_output(transform='pandas')
ct_ohe = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                                  (ohe,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')

pipe  =Pipeline([('CT',ct_ohe),('XGB',xgbm)])

In [25]:
pipe.get_params()

{'memory': None,
 'steps': [('CT',
   ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x0000015F288BEF00>),
                                   ('onehotencoder',
                                    OneHotEncoder(handle_unknown='ignore',
                                                  sparse_output=False),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x0000015F288BC5F0>)],
                     verbose_feature_names_out=False)),
  ('XGB',
   XGBRegressor(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=None, grow_policy=None, importance_type=None,
                interactio

In [26]:
params = {'XGB__n_estimators': [10,20,30,50,100,200],
         'XGB__max_depth': [2,3,4],
          'XGB__learning_rate': np.linspace(0.001,1,12)
          
         }

kfold = KFold(n_splits = 5,shuffle = True,random_state =24)

In [27]:
gcv = GridSearchCV(pipe,param_grid = params, cv = kfold, scoring = 'r2',verbose = 3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000015F288BEF00>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transfo...
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=24, ...))]),
             n_jobs=-1,
             param_grid={'XGB__learning_rate': array([0.001     , 0.09181818, 0.18263636, 0.27345455, 0.36427273,
       0.45509091, 0.54590909, 0.63672727, 0.72754545, 0.81836364,
       0.90918182, 1.        ]),
                         'XGB__max_depth': [2, 3, 4],
                         'XGB__n_estimators': [10, 20, 30, 50, 100, 200]},
             scoring='r2', verbose=3)

In [28]:
print(gcv.best_params_)
print(gcv.best_score_)

0.5000246021563131